In [1]:
# %pip install --upgrade tensorflow


In [2]:
import numpy as np
import tensorflow 
from tensorflow import keras
from keras.models import Sequential ,Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input,LSTM, GRU, Bidirectional, Conv1D, MaxPooling1D

from tensorflow.keras.utils import to_categorical


In [3]:
# Input and Output character set
input_chars = '123456789'
target_chars = '987654321'
num_samples = 1000
max_len = 5

# Mapping characters to integers
input_token_index = {char: i for i, char in enumerate(sorted(set(input_chars)))}
target_token_index = {char: i for i, char in enumerate(sorted(set(target_chars)))}

num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)

# Create dataset
encoder_input_data = np.zeros((num_samples, max_len, num_encoder_tokens))
decoder_input_data = np.zeros((num_samples, max_len, num_decoder_tokens))
decoder_target_data = np.zeros((num_samples, max_len, num_decoder_tokens))

for i in range(num_samples):
    input_seq = ''.join(np.random.choice(list(input_chars), max_len))
    target_seq = input_seq[::-1]

    for t, char in enumerate(input_seq):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_seq):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


In [4]:
latent_dim = 64  # Number of LSTM units

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Final Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 9)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 9)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 64),      │     18,944 │ input_layer[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     18,944 │ input_layer_1[0]… │
│                     │ 64), (None, 64),  │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 9)   │        585 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,473 (150.29 KB)

 Trainable params: 38,473 (150.29 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=20,
    validation_split=0.2
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.1044 - loss: 1.7526 - val_accuracy: 0.1670 - val_loss: 1.7379
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1666 - loss: 1.7347 - val_accuracy: 0.1760 - val_loss: 1.7228
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1840 - loss: 1.7199 - val_accuracy: 0.2200 - val_loss: 1.7166
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2825 - loss: 1.7187 - val_accuracy: 0.3440 - val_loss: 1.7160
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2288 - loss: 1.7160 - val_accuracy: 0.1930 - val_loss: 1.7030
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1975 - loss: 1.6958 - val_accuracy: 0.2270 - val_loss: 1.6752
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2244 - loss: 1.6588 - val_accuracy: 0.2560 - val_loss: 1.6350
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2449 - loss: 1.6145 - val_accuracy: 0.2910 - val_loss

In [6]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [7]:
def decode_sequence(input_seq):
    # Encode the input
    states_value = encoder_model.predict(input_seq)

    # Start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))

    # Initialize output string
    decoded_sentence = ''

    for _ in range(max_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = list(target_token_index.keys())[sampled_token_index]
        decoded_sentence += sampled_char

        # Update target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [8]:
test_input = '13579'
test_input_seq = np.zeros((1, max_len, num_encoder_tokens))
for t, char in enumerate(test_input):
    test_input_seq[0, t, input_token_index[char]] = 1.0

print("Input:", test_input)
print("Predicted Reverse:", decode_sequence(test_input_seq))


Input: 13579
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Reverse: 17733
